# Chapter 6: Planning

Key Takeaways:
- **Planning** allows agents to break down complex goals into manageable sub-tasks.
- **CrewAI** is a framework that orchestrates role-playing agents to work together.
- This pattern separates the *what* (requirements) from the *how* (execution plan).

### Heuristic: *Measure twice, cut once.*

## Setup and Initialization

In [ ]:
import os
import getpass
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

In [ ]:
# --- Configuration ---
if not os.getenv("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API key: ")

try:
    # Initialize Gemini model
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
    print(f"✅ Language model initialized: {llm.model}")
except Exception as e:
    print(f"🛑 Error initializing language model: {e}")
    llm = None

## Define Agent

We define a 'Planner & Writer' agent responsible for researching and summarizing a topic.

In [ ]:
# --- Define Agent ---
if llm:
    # Define a clear and focused agent
    planner_writer_agent = Agent(
        role='Article Planner and Writer',
        goal='Plan and then write a concise, engaging summary on a specified topic.',
        backstory=(
            'You are an expert technical writer and content strategist. '
            'Your strength lies in creating a clear, actionable plan before writing, '
            'ensuring the final summary is both informative and easy to digest.'
        ),
        verbose=True,
        allow_delegation=False,
        llm=llm 
    )

## Define Task

We give the agent a structured task: first plan, then write.

In [ ]:
# --- Define Task ---
topic = "The importance of Reinforcement Learning in AI"

high_level_task = Task(
    description=(
        f"1. Create a bullet-point plan for a summary on the topic: '{topic}'.\n"
        f"2. Write the summary based on your plan, keeping it around 200 words."
    ),
    expected_output=(
        "A final report containing two distinct sections:\n\n"
        "### Plan\n"
        "- A bulleted list outlining the main points of the summary.\n\n"
        "### Summary\n"
        "- A concise and well-structured summary of the topic."
    ),
    agent=planner_writer_agent,
)

## Execution

We form a `Crew` (even with a single agent) to manage the process execution.

In [ ]:
# --- Execute Crew ---
if llm:
    # Create the crew with a clear process
    crew = Crew(
        agents=[planner_writer_agent],
        tasks=[high_level_task],
        process=Process.sequential,
    )

    print("## Running the planning and writing task ##")
    result = crew.kickoff()
    
    print("\n\n---\n## Task Result ##\n---")
    print(result)

## Conclusion

By forcing the model to generate a plan *before* the final content, we often get higher quality and more structured outputs.